In [133]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation
from sklearn.datasets import fetch_20newsgroups
from time import time

In [376]:
from collections import Counter

In [155]:
import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()

In [95]:
import string
import pandas as pd
import numpy as np

In [324]:
from save_load import save_obj,load_obj

In [275]:
n_samples = len(data_samples)  # 2000
n_features = 1000
n_components = 10
n_top_words = 20

## Load Data generated from `clean_corpus.ipynb`.

In [310]:

mod_doc = load_obj('mod_doc')

In [311]:
mod_doc[0]

'sure story nad biased  disagree statement u.s. media  ruin israels reputation rediculous u.s. media  pro israeli medium world live europe  realize incidence describe  letter occur u.s. medium try  ignore u.s. subsidize israels existance  europeans degree think  reason report clearly  atrocity  shame austria daily report  inhuman act commit israeli soldier blessing  receive government holocaust guilt  away look jews treat race  power unfortunate '

In [319]:
tf_vectorizer = CountVectorizer(max_df=0.95, min_df=10, 
                                strip_accents = 'unicode',
                                max_features=5000,
                                stop_words='english', lowercase=True,
                                token_pattern=r'\b[a-zA-Z]{3,}\b')

In [320]:
t0 = time()
# tf = tf_vectorizer.fit_transform(mod_doc)
tf = tf_vectorizer.fit_transform(mod_doc)
print("done in %0.3fs." % (time() - t0))
print()

done in 2.378s.



In [321]:
# https://scikit-learn.org/stable/auto_examples/applications/plot_topics_extraction_with_nmf_lda.html#sphx-glr-auto-examples-applications-plot-topics-extraction-with-nmf-lda-py
def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        message = "Topic #%d: " % topic_idx
        message += " ".join([feature_names[i]
                             for i in topic.argsort()[:-n_top_words - 1:-1]])
        print(message)
    print()

In [422]:
def rank_words(vectorizer,tf):
    word_counts = tf.sum(axis=1)
    word_counts = np.asarray(word_counts.ravel())[0].tolist()
    words = tf_vectorizer.get_feature_names()
    
    words_list = list(zip(words, word_counts))
    
    return sorted(words_list, key= lambda t : t[1], reverse=True)

In [405]:
def common_words(vectorizer):
    words_dict = tf_vectorizer.vocabulary_
    return sorted(words_dict.items(), key= lambda t : t[1], reverse=True)

In [406]:
common_words(tf_vectorizer)[:10]

[('zoom', 4999),
 ('zone', 4998),
 ('zip', 4997),
 ('zionist', 4996),
 ('zionism', 4995),
 ('zero', 4994),
 ('zealand', 4993),
 ('yzerman', 4992),
 ('yup', 4991),
 ('youth', 4990)]

In [398]:
top_ten = rank_words(tf_vectorizer, tf)[:10]

In [399]:
top_ten 

[('highly', 4102),
 ('abs', 3909),
 ('score', 3901),
 ('santa', 3650),
 ('table', 3551),
 ('interface', 3336),
 ('desirable', 2792),
 ('lisp', 2055),
 ('hotel', 2035),
 ('friendly', 1896)]

In [407]:
lda = LatentDirichletAllocation(n_components=20, max_iter=5,
                                learning_method='online',
                                learning_offset=50.,
                                random_state=0)
t0 = time()
lda.fit(tf)
print("done in %0.3fs." % (time() - t0))

print("\nTopics in LDA model:")
tf_feature_names = tf_vectorizer.get_feature_names()
print_top_words(lda, tf_feature_names, 10)

done in 27.367s.

Topics in LDA model:
Topic #0: file use program window windows run version image display color
Topic #1: people gun right state law israel government kill war country
Topic #2: armenian armenians turkish people turkey russian kill woman greek armenia
Topic #3: president congress vote fbi house report press myers white koresh
Topic #4: edu com information mail send list available file include address
Topic #5: space launch nasa satellite earth air orbit mission program project
Topic #6: key use government encryption chip public clipper security law privacy
Topic #7: car use power ground engine good wire speed light high
Topic #8: god jesus believe christian people bible religion church man life
Topic #9: game team play player win season hockey league period score
Topic #10: bike keyboard motorcycle ride rider left turn shift won rid
Topic #11: disease study medical health offer condition patient sale drug new
Topic #12: know think use like good point post question peop

In [408]:
pyLDAvis.sklearn.prepare(lda, tf, tf_vectorizer)

/Users/thein/anaconda3/lib/python3.6/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
12    -0.177246 -0.049466       1        1  12.338208
17    -0.127152 -0.080137       2        1  11.140705
1     -0.081697 -0.208800       3        1   9.355782
0     -0.104786  0.169732       4        1   9.185256
4     -0.006933  0.056487       5        1   8.949424
8     -0.072962 -0.234838       6        1   7.189337
13    -0.158361  0.157716       7        1   6.717257
15    -0.117593  0.053949       8        1   5.065834
6     -0.042601 -0.067094       9        1   4.594386
7     -0.080212  0.172571      10        1   4.001481
9      0.233013  0.017904      11        1   3.860224
5      0.056571  0.024997      12        1   3.466631
2      0.052810 -0.169052      13        1   3.213588
3      0.042321 -0.147983      14        1   2.303879
11     0.124034  0.052867      15        1   2.132515
14    -0.118178  0.126466      16        1   2.090543
18     0.033234  0.119248      17        1   1.560473
16     0.068434 -0.173947      18        1   1.199543
19     0.384416  0.056951      19        1   1.010364
10     0.092888  0.122428      20        1   0.624569, topic_info=     Category         Freq        Term        Total  loglift  logprob
term                                                                 
2766  Default  4561.000000         max  4561.000000  30.0000  30.0000
3497  Default  2324.000000     problem  2324.000000  29.0000  29.0000
4744  Default  6872.000000         use  6872.000000  28.0000  28.0000
1937  Default  1999.000000         god  1999.000000  27.0000  27.0000
2489  Default  1632.000000         key  1632.000000  26.0000  26.0000
1869  Default  1363.000000        game  1363.000000  25.0000  25.0000
4201  Default  1356.000000       space  1356.000000  24.0000  24.0000
1731  Default  2562.000000        file  2562.000000  23.0000  23.0000
1408  Default  1664.000000       drive  1664.000000  22.0000  22.0000
1454  Default  2402.000000         edu  2402.000000  21.0000  21.0000
655   Default  1073.000000         car  1073.000000  20.0000  20.0000
3264  Default  4096.000000      people  4096.000000  19.0000  19.0000
4659  Default  2034.000000         try  2034.000000  18.0000  18.0000
4524  Default  3776.000000       think  3776.000000  17.0000  17.0000
4456  Default  1027.000000        team  1027.000000  16.0000  16.0000
4553  Default  3510.000000        time  3510.000000  15.0000  15.0000
2509  Default  4529.000000        know  4529.000000  14.0000  14.0000
4938  Default  2863.000000        work  2863.000000  13.0000  13.0000
1941  Default  4167.000000        good  4167.000000  12.0000  12.0000
3357  Default   969.000000        play   969.000000  11.0000  11.0000
3513  Default  2020.000000     program  2020.000000  10.0000  10.0000
657   Default  1035.000000        card  1035.000000   9.0000   9.0000
469   Default   482.000000        bike   482.000000   8.0000   8.0000
1948  Default  1316.000000  government  1316.000000   7.0000   7.0000
3888  Default   596.000000        rule   596.000000   6.0000   6.0000
3457  Default   691.000000   president   691.000000   5.0000   5.0000
2432  Default   894.000000       jesus   894.000000   4.0000   4.0000
1922  Default   490.000000         giz   490.000000   3.0000   3.0000
261   Default   654.000000    armenian   654.000000   2.0000   2.0000
459   Default   481.000000         bhj   481.000000   1.0000   1.0000
...       ...          ...         ...          ...      ...      ...
3667  Topic20    44.906777     rebuild    48.398232   5.0010  -4.6540
3858  Topic20    25.884561      rogers    28.701161   4.9726  -5.2050
2490  Topic20   303.078673    keyboard   354.552489   4.9190  -2.7446
3826  Topic20   162.021047        ride   212.750324   4.8035  -3.3709
4324  Topic20    27.775282      stroke    37.555431   4.7742  -5.1345
153   Topic20    22.081256         ama    30.650696   4.7479  -5.3639
4067  Topic20    86.579732       shift 

In [455]:
tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=50, 
                                   max_features=5000,
                                   stop_words='english', lowercase=True)

In [456]:
t0 = time()
# tf = tf_vectorizer.fit_transform(mod_doc)
tfidf = tfidf_vectorizer.fit_transform(mod_doc)
print("done in %0.3fs." % (time() - t0))
print()

done in 2.777s.



In [457]:
common_words(tfidf_vectorizer)[:10]

[('zoom', 4999),
 ('zone', 4998),
 ('zip', 4997),
 ('zionist', 4996),
 ('zionism', 4995),
 ('zero', 4994),
 ('zealand', 4993),
 ('yzerman', 4992),
 ('yup', 4991),
 ('youth', 4990)]

In [458]:
rank_words(tfidf_vectorizer, tfidf)[:10]

[('interface', 15.64036715885867),
 ('table', 14.360831025528324),
 ('bed', 14.208717306092124),
 ('highly', 14.047837349802867),
 ('desirable', 13.919189819005037),
 ('military', 13.866216206038965),
 ('answer', 13.80428634018271),
 ('relief', 13.7998117236344),
 ('abs', 13.688291320345177),
 ('micro', 13.033006118050238)]

In [459]:
lda_idf = LatentDirichletAllocation(n_components=20, max_iter=5,
                                learning_method='batch',
                                learning_offset=10.,
                                random_state=0)
t0 = time()
lda_idf.fit_transform(tfidf)
print("done in %0.3fs." % (time() - t0))

print("\nTopics in LDA model:")
tfidf_feature_names = tfidf_vectorizer.get_feature_names()
print_top_words(lda_idf, tfidf_feature_names, 10)

done in 18.582s.

Topics in LDA model:
Topic #0: 00 sale condition offer shipping sell new good battery manual
Topic #1: space nasa year launch president think orbit people use 1993
Topic #2: msg gordon pitt surrender banks geb intellect cadre shameful chastity
Topic #3: lc list mailing edu hello thank adams ii jack information
Topic #4: game team play hockey playoff season player nhl leafs year
Topic #5: jews people think jewish know israel islam koresh good right
Topic #6: tank think plant water guess gas trial brian game regards
Topic #7: drive good car year use like know think time new
Topic #8: jet auto film vs old wear spare like pain chemical
Topic #9: government key gun people right law use encryption state phone
Topic #10: god people jesus think believe know christian bible life thing
Topic #11: key chip fbi warrant batf test escrow clipper video card
Topic #12: armenian armenians turkish turkey armenia turks genocide massacre serdar soviet
Topic #13: joke period pp lewis goal

In [460]:
t0 = time()
tfdif_vis = pyLDAvis.sklearn.prepare(lda_idf, tfidf, tfidf_vectorizer,sort_topics=False)
print("done in %0.3fs." % (time() - t0))

done in 3.079s.


In [461]:
tfdif_vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.158213  0.067128       1        1   2.578817
1      0.028005 -0.039046       2        1   8.278160
2      0.094225 -0.064251       3        1   1.797641
3     -0.095748  0.005832       4        1   1.499427
4      0.110575  0.158673       5        1   2.893168
5      0.126201 -0.064765       6        1   4.198822
6      0.072701  0.023324       7        1   1.557046
7     -0.024843  0.031826       8        1  19.471353
8      0.027947  0.040334       9        1   1.268768
9      0.060114 -0.089406      10        1  10.334192
10     0.103772 -0.073701      11        1  14.299242
11    -0.082661 -0.088737      12        1   1.433142
12     0.033163 -0.067055      13        1   2.597697
13     0.036145  0.002585      14        1   1.320574
14    -0.087278 -0.033065      15        1   1.158952
15    -0.099593 -0.014301      16        1   1.406537
16     0.078248  0.162347      17        1   1.619802
17    -0.055581  0.104616      18        1   1.680290
18    -0.030601 -0.046920      19        1   1.397085
19    -0.136578 -0.015418      20        1  19.209285, topic_info=     Category        Freq         Term       Total  loglift  logprob
term                                                                
2153  Default  145.000000        thank  145.000000  30.0000  30.0000
962   Default  116.000000         game  116.000000  29.0000  29.0000
1215  Default   89.000000          key   89.000000  28.0000  28.0000
415   Default   94.000000         card   94.000000  27.0000  27.0000
659   Default   51.000000       delete   51.000000  26.0000  26.0000
760   Default  129.000000          edu  129.000000  25.0000  25.0000
1226  Default  238.000000         know  238.000000  24.0000  24.0000
1314  Default   98.000000         mail   98.000000  23.0000  23.0000
455   Default   75.000000         chip   75.000000  22.0000  22.0000
2158  Default  212.000000        think  212.000000  21.0000  21.0000
2133  Default   79.000000         team   79.000000  20.0000  20.0000
986   Default  114.000000          god  114.000000  19.0000  19.0000
1274  Default  217.000000         like  217.000000  18.0000  18.0000
737   Default   71.000000       driver   71.000000  17.0000  17.0000
0     Default   52.000000           00   52.000000  16.0000  16.0000
988   Default  212.000000         good  212.000000  15.0000  15.0000
736   Default  119.000000        drive  119.000000  14.0000  14.0000
2367  Default  148.000000         year  148.000000  13.0000  13.0000
2075  Default   58.000000        stuff   58.000000  12.0000  12.0000
1450  Default  137.000000         need  137.000000  11.0000  11.0000
1047  Default   65.000000           hi   65.000000  10.0000  10.0000
1891  Default   42.000000         scsi   42.000000   9.0000   9.0000
1869  Default   55.000000         sale   55.000000   8.0000   8.0000
1632  Default  121.000000         post  121.000000   7.0000   7.0000
2331  Default   63.000000          win   63.000000   6.0000   6.0000
1606  Default   72.000000         play   72.000000   5.0000   5.0000
1803  Default   57.000000        reply   57.000000   4.0000   4.0000
1361  Default   29.000000          meg   29.000000   3.0000   3.0000
2250  Default  265.000000          use  265.000000   2.0000   2.0000
165   Default   63.000000      address   63.000000   1.0000   1.0000
...       ...         ...          ...         ...      ...      ...
1685  Topic20   64.657947      program   91.244384   1.3053  -5.0136
1559  Topic20   40.393628           pc   53.979218   1.3598  -5.4840
1991  Topic20   44.539207     software   60.557481   1.3426  -5.3863
232   Topic20   33.561146  application   44.667258   1.3639  -5.6693
998   Topic20   30.719370      graphic   40.215874   1.3804  -5.7578
1399  Topic20   28.211882        modem   36.424438   1.3943  -5.8429
2272  Topic20   41.913599      version   60.201893   1.2877  -5.4471
1397  Top

## For Single topic

In [226]:
ten_doc = np.array(list(map(spacy_tokenizer,data_samples[:10])))
ten_doc[0]

'sure story nad biased  disagree statement u.s. media  ruin israels reputation rediculous u.s. media  pro israeli medium world live europe  realize incidence describe  letter occur u.s. medium try  ignore u.s. subsidize israels existance  europeans degree think  reason report clearly  atrocity  shame austria daily report  inhuman act commit israeli soldier blessing  receive government holocaust guilt  away look jews treat race  power unfortunate '

In [244]:
ten_tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2, 
                                strip_accents = 'unicode',
                                max_features=3000,
                                stop_words='english', lowercase=True,
                                token_pattern=r'[a-zA-Z\-][a-zA-Z\-]{2,}')
ten_tf = ten_tf_vectorizer.fit_transform(ten_doc)

In [245]:
lda = LatentDirichletAllocation(n_components=10, max_iter=5,
                                learning_method='online',
                                learning_offset=50.,
                                random_state=0)

In [246]:
lda.fit(ten_tf)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7,
             learning_method='online', learning_offset=50.0,
             max_doc_update_iter=100, max_iter=5, mean_change_tol=0.001,
             n_components=10, n_jobs=None, n_topics=None, perp_tol=0.1,
             random_state=0, topic_word_prior=None,
             total_samples=1000000.0, verbose=0)

In [247]:
ten_tf_feature_names = ten_tf_vectorizer.get_feature_names()
print_top_words(lda, ten_tf_feature_names, 5)

Topic #0: read accept maybe expect run
Topic #1: know table maybe like israeli
Topic #2: table use like treat product
Topic #3: end year price real product
Topic #4: expect biased change new treat
Topic #5: work like want year wonder
Topic #6: israel question like realize work
Topic #7: israeli government power jews look
Topic #8: let change right new certainly
Topic #9: real certainly like run right



In [200]:
print_top_words(lda, single_tf_feature_names, 5)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7,
             learning_method='online', learning_offset=50.0,
             max_doc_update_iter=100, max_iter=5, mean_change_tol=0.001,
             n_components=10, n_jobs=None, n_topics=None, perp_tol=0.1,
             random_state=0, topic_word_prior=None,
             total_samples=1000000.0, verbose=0)

In [238]:
data_samples[1]

"\n\n\n\n\n\n\nYeah, do you expect people to read the FAQ, etc. and actually accept hard\natheism?  No, you need a little leap of faith, Jimmy.  Your logic runs out\nof steam!\n\n\n\n\n\n\n\nJim,\n\nSorry I can't pity you, Jim.  And I'm sorry that you have these feelings of\ndenial about the faith you need to get by.  Oh well, just pretend that it will\nall end happily ever after anyway.  Maybe if you start a new newsgroup,\nalt.atheist.hard, you won't be bummin' so much?\n\n\n\n\n\n\nBye-Bye, Big Jim.  Don't forget your Flintstone's Chewables!  :) \n--\nBake Timmons, III"

In [248]:
ten_topic = lda.transform(ten_tf)

In [250]:
for n in range(ten_topic.shape[0]):
    topic_most_pr = ten_topic[n].argmax()
    print("doc: {} topic: {}\n".format(n,topic_most_pr))

doc: 0 topic: 7

doc: 1 topic: 0

doc: 2 topic: 6

doc: 3 topic: 3

doc: 4 topic: 8

doc: 5 topic: 0

doc: 6 topic: 9

doc: 7 topic: 1

doc: 8 topic: 2

doc: 9 topic: 5



In [251]:
print_top_words(lda, ten_tf_feature_names, 5)

Topic #0: read accept maybe expect run
Topic #1: know table maybe like israeli
Topic #2: table use like treat product
Topic #3: end year price real product
Topic #4: expect biased change new treat
Topic #5: work like want year wonder
Topic #6: israel question like realize work
Topic #7: israeli government power jews look
Topic #8: let change right new certainly
Topic #9: real certainly like run right



In [252]:
ten_doc[2]

'realize principle strong  point like know ask question  sort arab country  want continue think tank charade  fixation israel stop  start ask  sort question arab country  realize  work arab country treatment jews  decade bad fixation israel  begin look like biased attack  group recognize stupid center  policy research fancy bigot  hat israel'

In [255]:
(lda.transform(ten_tf_vectorizer.transform([ten_doc[0]]))).argmax()

7